### Erreur en position ``sparsity_level = 3`` & ``batch_size = 200``

In [1]:
import numpy as np
import pandas as pd
from numba import njit
from csc.atoms import *
from csc.dictionary import *
import matplotlib.pyplot as plt

# atom parameters
bmin = 0.005
bmax = 0.60
b_step = 0.005
b_values = np.arange(bmin, bmax, b_step)

ymin = 0.01
ymax = 0.034
y_step = 0.002
y_values = np.arange(ymin, ymax, y_step)

s_values = [1]

dictionary = ZSDictionary.from_values(b_values, y_values, s_values)
print('The dictionary has {} different atoms'.format(len(dictionary)))

The dictionary has 1428 different atoms


In [2]:
posErr_s3_b200 = dictionary.ompPositionErrorPipeline(sparsity_level=3, batch_size=200, cores=50, verbose=False)

OMP Test Batch for noise_level = 0.08:   0%|          | 0/200 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
plt.figure(figsize=(10, 6))

# Pre-processing
results_dict = posErr_s3_b200
# Génère des couleurs
colors = plt.cm.viridis(np.linspace(0, 1, len(results_dict)))  

# Définition des bins pour les erreurs de position
max_error = max(max(counter.keys()) for counter in results_dict.values())
min_error = min(min(counter.keys()) for counter in results_dict.values())
bins = np.arange(min_error, max_error + 50, 50)  # Création des bins de 50 unités

# Parcourir le dictionnaire de résultats et dessiner un histogramme pour chaque niveau de bruit
for (noise_level, counter), color in zip(results_dict.items(), colors):
    # Regroupement des erreurs par bins
    errors, counts = zip(*counter.items())
    summed_counts, _ = np.histogram(errors, bins=bins, weights=counts)
    
    # Normaliser pour faire un histogramme de densité
    summed_counts = summed_counts / sum(summed_counts)

    # Dessiner l'histogramme de densité
    plt.bar(bins[:-1], summed_counts, color=color, alpha=0.8, label=f'Noise {noise_level:.2f}', width=45)

plt.title('3 atoms/signal & 20 signals/batch')
plt.xlabel('Position Error')
plt.ylabel('Density')
plt.xticks(bins)  # Définir les ticks sur les bins
plt.legend(title="Noise Levels")
plt.grid(True)
plt.show()